# **01.2 - Préparation des données pour l’étape 3 (Super-Images)**

Ce notebook prépare les données nécessaires à l’étape 3, qui consiste à construire des super-images 3×3 à partir de 9 frames par vidéo.

Objectifs :

- Extraire 9 frames uniformément espacées pour chaque vidéo.

- Nettoyer ces frames (floues / doublons).

- Générer des splits 5-fold multi-label adaptés au dataset `9_forstep3`

L’ensemble des scripts utilisés se trouve dans src/preprocess/.

In [ ]:
# Si vous êtes sur Google Colab :
# from google.colab import drive
# drive.mount('/content/drive')

# On se place à la racine du projet
%cd /content/qv-pipe-classifier

import os
print("Dossier courant :", os.getcwd())


## 1 - Vérification des scripts nécessaires

Les scripts suivants doivent être présents dans `src/preprocess/`:

- `extract_9frames_uniform.py`
- `clean_frames_9.py`
- `make_splits_5fold_9frames.py`


In [ ]:
!ls -R src/preprocess

## 2 - Extraction de 9 frames uniformes par vidéo

Pour la construction des super-images, chaque vidéo sera représentée par 9 frames régulièrement espacées entre le début et la fin.

Les frames extraites seront placées dans :

```text
data/frames/9_forstep3/
    videoid_f00.jpg
    videoid_f01.jpg
    ...
    videoid_f08.jpg

L’extraction est assurée par : src/preprocess/extract_9frames_uniform.py


In [ ]:
RUN_EXTRACTION_9 = True  # Set to False if already done

if RUN_EXTRACTION_9:
    import subprocess, sys

    print(">> Extracting 9 frames per video for Step 3...")
    result = subprocess.run(
        [sys.executable, "-m", "src.preprocess.extract_9frames_uniform"],
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    print(result.stderr)
else:
    print("Extraction (9 frames) skipped.")

## 3 - Nettoyage des frames extraites

Les frames extraites peuvent contenir :

- des images floues,

- des images doublons.

Ce nettoyage est important pour éviter d’utiliser des frames inutiles lors de la construction des super-images.

Les frames rejetées seront déplacées dans :

```text
data/frames/9_forstep3_removed/
    blurred/
    duplicates/
```

Le nettoyage est assuré par: src/preprocess/clean_frames_9.py.


In [ ]:
RUN_CLEANING_9 = True  # Mettre sur False si déjà fait

if RUN_CLEANING_9:
    import subprocess, sys

    print(">> Nettoyage des frames (floues + doublons)...")
    result = subprocess.run(
        [sys.executable, "-m", "src.preprocess.clean_frames_9"],
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    print(result.stderr)
else:
    print("Nettoyage ignoré.")


## 4 - Génération des splits 5-fold (pour 9_forstep3)

Pour entraîner les modèles super-image en cross-validation, on génère un split 5-fold stratifié pour un problème multi-label.

Cette étape :

1. charge le JSON des labels : `data/labels/track1-qv_pipe_train.json`

2. crée les folds multi-label stratifiés

3. applique ces folds aux frames du dossier `9_forstep3`

4. génère les fichiers suivants :

 - `data/splits/video_folds_5fold.csv`
 - `data/splits/frames_9_forstep3_folds.csv`
 - `reports/tables/preprocessing/class_distribution_per_fold_step3.csv`


In [ ]:
RUN_SPLITS_9 = True  # Mettre sur False si les splits existent déjà

if RUN_SPLITS_9:
    import subprocess, sys

    print(">> Génération des splits 5-fold pour 9_forstep3...")
    result = subprocess.run(
        [sys.executable, "-m", "src.preprocess.make_splits_5fold_9frames"],
        capture_output=True,
        text=True,
    )
    print(result.stdout)
    print(result.stderr)
else:
    print("Génération des splits ignorée.")


## 5 - Vérifications rapides

On vérifie que :

- les fichiers CSV existent,

- les folds sont bien remplis,

- les chemins de frames sont corrects.

In [ ]:
from pathlib import Path
import pandas as pd

frames9_csv = Path("data/splits/frames_9_forstep3_folds.csv")
video_csv   = Path("data/splits/video_folds_5fold.csv")

print("CSV frames 9_forstep3 présent :", frames9_csv.exists())
print("CSV vidéo (folds) présent     :", video_csv.exists())

if frames9_csv.exists():
    df_frames9 = pd.read_csv(frames9_csv)
    print("\nAperçu du CSV (frames 9_forstep3) :")
    display(df_frames9.head())

    print("\nNombre de frames par fold :")
    print(df_frames9["fold"].value_counts().sort_index())

if video_csv.exists():
    df_videos = pd.read_csv(video_csv)
    print("\nNombre de vidéos par fold :")
    print(df_videos["fold"].value_counts().sort_index())


In [10]:
!ls /content


qv-pipe-classifier  sample_data
